In [645]:
#!/usr/bin/python
import numpy as np
import scipy.sparse
import pickle
import xgboost as xgb

### simple example
# load file from text file, also binary buffer generated by xgboost
# dtrain = xgb.DMatrix('./demo/rank_aida/mq2008.train')

# dtest = xgb.DMatrix('./demo/rank_aida/mq2008.test')

In [547]:
print('start running example of build DMatrix from scipy.sparse CSR Matrix')
labels = []
row = []; col = []; dat = []
i = 0
for l in open('./demo/rank_aida/mq2008.train'):
    arr = l.split()
    labels.append(int(arr[0]))
    for it in arr[1:]:
        k,v = it.split(':')
        row.append(i); col.append(int(k)); dat.append(float(v))
    i += 1
csr = scipy.sparse.csr_matrix((dat, (row, col)))
dtrain = xgb.DMatrix(csr, label=labels)

start running example of build DMatrix from scipy.sparse CSR Matrix


In [548]:
print('start running example of build DMatrix from scipy.sparse CSR Matrix')
labels = []
row = []; col = []; dat = []
i = 0
for l in open('./demo/rank_aida/mq2008.test'):
    arr = l.split()
    labels.append(int(arr[0]))
    for it in arr[1:]:
        k,v = it.split(':')
        row.append(i); col.append(int(k)); dat.append(float(v))
    i += 1
csr = scipy.sparse.csr_matrix((dat, (row, col)))
dtest = xgb.DMatrix(csr, label=labels)

start running example of build DMatrix from scipy.sparse CSR Matrix


In [646]:
import redis
import json
conn = redis.Redis(password='unswkg', host='43.240.96.216', port=8379, decode_responses=True)

def get_feature_results(exp_id, d_t, data_source='conll'):
    fea_res = conn.lrange('result:::::' + data_source + ':::::' + str(exp_id) + ':::::' + str(d_t), 0, -1)
    fea_vecs = [json.loads(res) for res in fea_res]
    return fea_vecs

In [671]:
def fetch_all_features(exp_id, data_type):
    res_feas = get_feature_results(exp_id, data_type)
    res_feas_ids = [[res[0].strip('(').split(', ')[0]] + res[1:-1] + [res[-1]] for res in res_feas]
    res_feas_ids = np.array(res_feas_ids, dtype=np.float64)
    return res_feas_ids

In [672]:
def trans_data(data):
#     print(data)
    d_np = data[:, 1:-1]
#     d_np = np.insert(d_np, 0, 0, axis=1)
    print(d_np)
    d_labels = data[:, -1]
    print(d_labels)
    
    idxs = np.where(d_labels == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_labels)) - idxs
    print(d_groups)
    xgb_data = xgb.DMatrix(data=d_np, label=d_labels)
    xgb_data.set_group(d_groups)
    return xgb_data

In [673]:
def evalerror(preds, dt, d_size):
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
    matched_ids = []
    q_id = 0
    for x in d_groups:
        pre_res = preds[q_id: x + q_id]
        if(preds[q_id] == max(pre_res)):
            if len([x for x in pre_res if x == preds[q_id]]) == 1:
                matched_ids.append(q_id)
        q_id += x
    return float(len(matched_ids)) / d_size, len(matched_ids)

In [674]:
d_train = fetch_all_features('basic_feature_phrases_tf_idf_entropy', 'train')
d_test = fetch_all_features('basic_feature_phrases_tf_idf_entropy', 'testb')

In [675]:
dtrain = trans_data(d_train)
dtest = trans_data(d_test)
# print(dtrain_g)
# print(dtest_g)

[[  2.42454797e-01   1.10000000e+02   5.16917646e-01 ...,   1.18103223e-02
   -1.88688687e-02  -3.93216626e-02]
 [  5.16917646e-01   1.10000000e+02   5.16917646e-01 ...,   1.44081062e-02
   -2.36827386e-02  -4.90766352e-02]
 [  1.11083105e-01   1.10000000e+02   5.16917646e-01 ...,   1.20827537e-02
   -2.31386486e-02  -4.66195834e-02]
 ..., 
 [  1.57753588e-04   1.17000000e+02   9.52989459e-01 ...,   1.92030027e-02
    1.37146913e-02   1.00558170e-02]
 [  1.57753588e-04   1.17000000e+02   9.52989459e-01 ...,   1.14426007e-02
    1.22668194e-03  -5.58393058e-03]
 [  1.57753588e-04   1.17000000e+02   9.52989459e-01 ...,   9.14815535e-03
   -3.73305784e-03  -1.23205333e-02]]
[ 1.  0.  0. ...,  0.  0.  0.]
[50 24 50 ..., 50 50 50]
[[  9.63790298e-01   7.90000000e+01   9.63790298e-01 ...,   5.16915922e-02
    8.95827305e-03  -1.95306064e-02]
 [  1.22580649e-02   7.90000000e+01   9.63790298e-01 ...,   4.42164690e-02
   -1.64660165e-02  -5.69210069e-02]
 [  4.67741955e-03   7.90000000e+01   9.

In [676]:
param = {'max_depth':6, 'eta':0.01, 'silent':0, 'objective':'rank:pairwise', 'min_child_weight':0.01}

In [679]:
num_round = 1000
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtrain)
a = evalerror(preds, dtrain, 18541)
print("The accuracy for training data is {}, the correct num is {}".format(a[0], a[1]))

The accuracy for training data is 0.9255703575858907, the correct num is 17161


In [680]:
preds = bst.predict(dtest)
a = evalerror(preds, dtest, 4485)
print("The accuracy for testing data is {}, the correct num is {}".format(a[0], a[1]))

The accuracy for testing data is 0.861984392419175, the correct num is 3866


In [687]:
n_estimators = range(100, 5000, 400)
max_depth = [4, 6, 8, 10]

In [689]:
for x in n_estimators:
    num_round = x
    for dep in max_depth:
        param = {'max_depth':dep, 'eta':0.01, 'silent':0, 'objective':'rank:pairwise', 'min_child_weight':0.01}
        bst = xgb.train(param, dtrain, num_round)
        preds = bst.predict(dtrain)
        a = evalerror(preds, dtrain, 18541)
        print("n_estimators: {}, max_depth: {}, acc_training: {}, corr_num: {}".format(num_round, dep, a[0], a[1]))
        preds = bst.predict(dtest)
        a = evalerror(preds, dtest, 4485)
        print("n_estimators: {}, max_depth: {}, acc_testing: {}, corr_num: {}".format(num_round, dep, a[0], a[1]))
        print("---------------------------------")

n_estimators: 100, max_depth: 4, acc_training: 0.7684051561404455, corr_num: 14247
n_estimators: 100, max_depth: 4, acc_testing: 0.7154960981047938, corr_num: 3209
---------------------------------
n_estimators: 100, max_depth: 6, acc_training: 0.8289736260180142, corr_num: 15370
n_estimators: 100, max_depth: 6, acc_testing: 0.7906354515050167, corr_num: 3546
---------------------------------
n_estimators: 100, max_depth: 8, acc_training: 0.8812901138018445, corr_num: 16340
n_estimators: 100, max_depth: 8, acc_testing: 0.8118171683389075, corr_num: 3641
---------------------------------
n_estimators: 100, max_depth: 10, acc_training: 0.9205544469014616, corr_num: 17068
n_estimators: 100, max_depth: 10, acc_testing: 0.8256410256410256, corr_num: 3703
---------------------------------
n_estimators: 500, max_depth: 4, acc_training: 0.8477428401920069, corr_num: 15718
n_estimators: 500, max_depth: 4, acc_testing: 0.8055741360089186, corr_num: 3613
---------------------------------
n_estima

In [691]:
n_est = [x for x in n_estimators]
train_scores = np.zeros(shape=(len(n_est), len(max_depth)))
test_scores = np.zeros(shape=(len(n_est), len(max_depth)))